In [1]:
# result = resumes_collection.delete_many({})

In [16]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# # API keys
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from a .env file
uri = os.environ.get('MONGODB_URI')
api_key = os.environ.get('OPENAI_API_KEY')

In [22]:
%pip install pymongo  

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import certifi
import pymongo  #this is a library to allows us to work with mongodb from python
from pymongo.mongo_client import MongoClient   #this is used to create a connection to a mongodb instance
from pymongo.server_api import ServerApi    #is used to specify the version of mongodb server API to use when connecting
import json
from openai import OpenAI
from datetime import datetime, timezone
from pymongo.errors import ConfigurationError

In [24]:
# #URI is (Uniform Resource Identifier)
# # we used username , password , cluster address and connection options like (retryWrites=true, w=majority, appName=Cluster0)


# Create a new client and connect to the server
# client = MongoClient(uri, server_api=ServerApi('1'), tlsCAFile=certifi.where()),
# server_api=ServerApi('1'), tlsCAFile=certifi.where()


# # Send a ping to confirm a successful connection and printing a success message or an error it it fails
import os
from pymongo import MongoClient
from pymongo.server_api import ServerApi
import certifi
from pymongo.errors import ConfigurationError, ConnectionFailure
import sys
import os
import pymongo
import certifi
from pymongo import MongoClient
from pymongo.server_api import ServerApi

import os
import pymongo
import certifi
from pymongo import MongoClient
from pymongo.server_api import ServerApi

# Get MongoDB URI from environment variables
uri = os.environ.get('MONGODB_URI')
if not uri:
    raise ValueError("MongoDB URI is not set in environment variables")

print("Attempting MongoDB Connection...")

client = MongoClient(
    uri, 
    server_api=ServerApi('1'),  # Ensure this line has a comma at the end
    tlsCAFile=certifi.where(),
    connectTimeoutMS=30000,
    socketTimeoutMS=30000
)

# Ping the deployment
client.admin.command('ping')
print("Successfully connected to MongoDB!")



# # Explicitly load environment variables
# # load_dotenv()

# # # Print environment details
# # print("Python Version:", sys.version)
# # print("Current Working Directory:", os.getcwd())
# # print("Environment Variables:")
# # print("MONGODB_URI:", os.getenv('MONGODB_URI'))

# try:
#     import pymongo

#     # Detailed connection attempt
#     uri = os.getenv('MONGODB_URI')
#     if not uri:
#         raise ValueError("MongoDB URI is not set in environment variables")

#     print("Attempting MongoDB Connection...")
#     client = MongoClient(
#         uri, 
#         server_api=ServerApi('1'), 
#         tlsCAFile=certifi.where(),
#         connectTimeoutMS=30000,
#         socketTimeoutMS=30000
#     )

#     # Ping the deployment
#     client.admin.command('ping')
#     print("Successfully connected to MongoDB!")

# except Exception as e:
#     print(f"Full Error Details:")
#     print(f"Error Type: {type(e).__name__}")
#     print(f"Error Message: {str(e)}")
#     import traceback
#     traceback.print_exc()

Attempting MongoDB Connection...
Successfully connected to MongoDB!



NEW CODE


In [5]:
%pip install pymongo python-dotenv openai PyPDF2 python-docx langchain_community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install PyPDF2 
%pip install python-docx  

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip install --upgrade openai


  Attempting uninstall: openai
    Found existing installation: openai 1.60.1
    Uninstalling openai-1.60.1:
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\python312\\scripts\\openai.exe'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



## **Resume Extraction and Processing System**

In [8]:
# These lines access the 'ResumeDatabase' and its 'Resume' collection
db = client['ResumeDatabase']
#
resumes_collection = db['Resume']
#Retrieves a specific collection named 'Resume' from the 'ResumeDatabase'.


# In mongodb atlas, project
# cluster
# configuratuion

# Fetch only the latest document from 'Resume' collection with sorted by "uploadedAt" field in decending order(-1)
latest_resume = resumes_collection.find_one(sort=[('uploadedAt', -1)])
#



if latest_resume:  # this checks if a resume was found in the database
    print("Found the latest resume in the database.")

    filename = latest_resume.get('filename', 'Unknown')
    #provides file name if not displays "UNKNOWN"
    content = latest_resume.get('content')
    #extract the content of document
    content_type = latest_resume.get('contentType')
    #content type give info about nature and format of document  like PDF"application/pdf
#or .doc and .docx

    print(f"Processing: {filename}") #This prints the filename of the resume being processed.


    if content_type == 'application/pdf':
      #this check if a document is a pdf
        # Process PDF
        try:
            import io
            from PyPDF2 import PdfReader

            pdf_file = io.BytesIO(content) #These lines create a file-like object from the content and initialize a PDF reader.
            pdf_reader = PdfReader(pdf_file)


             #This extracts text from each page of the PDF and concatenates it.
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()

            print(f"Extracted text from PDF (Resume Text): {text[:]}...")
        except Exception as e:
            print(f"Error processing PDF {filename}: {str(e)}")

    elif content_type in ['application/msword', 'application/vnd.openxmlformats-officedocument.wordprocessingml.document']:
        # This checks if the document is a Word file (.doc or .docx).

        try:
            import io
            from docx import Document

            #These lines create a file-like object from the content and load it as a Word document.
            doc_file = io.BytesIO(content)
            document = Document(doc_file)

            #This extracts text from each paragraph in the Word document and joins them with newlines.
            text = "\n".join([para.text for para in document.paragraphs])

            print(f"Extracted text from Word document (Resume Text): {text[:]}...") # this prints the extracted text frm the word document
        except Exception as e:
            print(f"Error processing Word document {filename}: {str(e)}") #this handles cases where the file type is neither PDF nor word

    else:
        print(f"Unsupported file type: {content_type}")

    print("------------------------")

else:
    print("No resumes found in the database.")

print("Processing completed.")

NameError: name 'client' is not defined

# **Providing the resume to openai api**

In [11]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [12]:
#The OpenAI class is used to interact with OpenAI API
import json
#The datetime class is used for timestamping
from datetime import datetime

#This line takes two parameters:the resume text and an API key for OpenAI.
def analyze_resume_with_openai(text, api_key):
  #This creates an instance of the OpenAI client using the provided API key.
  client = OpenAI(api_key=api_key)

#This block sends a request to the OpenAI API to analyze the resume using GTP-3.5 turbo
  try:
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
        messages=[
            {
                # "role": "system",
                # "content": '''You are a resume analyzer. Analyze the following resume and provide key insights, improvements and recomendations. Do not include any concluding statements like 'Overall,' 'In conclusion,' or
                #     'By implementing these improvements'. Focus only on presenting insights, improvements, and specific recommendations.'''
        
                "role": "system", 
                "content": '''You are a resume analyzer. Analyze the following resume and provide a summary in one paragraph under 500 words. The summary should highlight the individual's qualifications, experience, and key strengths, and explicitly make all skills the bold( write them inside **). Give it without any title'''
                # "content": '''You are a resume analyzer. Analyze the following resume and provide Summary under 500 words in one paragraph also make the skills bold.'''

            },
            {
                "role": "user",
                "content": text
            }
        ],
        # this controls the randomness of the model output , higher temperature means increase creativity and variability
        temperature=0
    )

      # this returns the analysis result or an error message if an exception occurs
    return response.choices[0].message.content

  except Exception as e:
      return f"Error analyzing resume: {str(e)}"


# NEW Highlight code
def get_highlight_sections(analysis_text, api_key):
    client = OpenAI(api_key=api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": """You are a resume improvement expert. Analyze the resume sections that need improvement and provide specific suggestions for enhancement. Your response MUST be a valid JSON string in this format:
{
    "sections": [
        {
            "text": "exact texts from resume that needs improvement",
            "reason": "specific reason why this needs improvement",
            "suggestion": "detailed suggestion for how to rewrite this section"
        }
    ]
}
Do not include anything outside of the JSON structure."""
                },
                {
                    "role": "user",
                    "content": f"Identify sections needing improvement and provide specific suggestions for enhancement: {analysis_text}"
                }
            ],
            temperature=0
        )
        
        try:
            content = response.choices[0].message.content.strip()
            highlights = json.loads(content)
            if not isinstance(highlights, dict) or 'sections' not in highlights:
                highlights = {"sections": []}
            return highlights
        except json.JSONDecodeError:
            print("Error parsing JSON response")
            return {"sections": []}
            
    except Exception as e:
        print(f"Error in get_highlight_sections: {str(e)}")
        return {"sections": []}





# commented out bcz we used this method earlier
# # Connect to MongoDB and fetch resume
# db = client['ResumeDatabase']
# resumes_collection = db['Resume']

# # Fetch latest resume
# latest_resume = resumes_collection.find_one(sort=[('uploadedAt', -1)])

if latest_resume:# this line checks if the resume was found in the database
  content = latest_resume.get('content')
  #these lines extract the content and content type of the resume from the databse document
  content_type = latest_resume.get('contentType')

  # Extract text based on file type
  #This block handles PDF files. It uses PyPDF2 to read the PDF content and extract text from each page.
  text = ""
  if content_type == 'application/pdf':
      try:
          import io
          from PyPDF2 import PdfReader

          pdf_file = io.BytesIO(content)
          pdf_reader = PdfReader(pdf_file)
          for page in pdf_reader.pages:
              text += page.extract_text()
      except Exception as e:
          print(f"Error processing PDF: {str(e)}")

#This block handles Word documents (.doc and .docx). It uses the python-docx library to read the document and extract text from each paragraph.
  elif content_type in ['application/msword', 'application/vnd.openxmlformats-officedocument.wordprocessingml.document']:
      try:
          import io
          from docx import Document

          doc_file = io.BytesIO(content)
          document = Document(doc_file)
          text = "\n".join([para.text for para in document.paragraphs])
      except Exception as e:
          print(f"Error processing Word document: {str(e)}")

  if text:# if text was successfully extracted
      # Send to OpenAI for analysis
      analysis = analyze_resume_with_openai(text, api_key)
      highlights = get_highlight_sections(text, api_key)


      # Store the analysis back in MongoDB
      #By storing resume in database it can be accessed later without re-analyze the resume, this is cheap option also bcz user does not have to pay extra for analyzed the resume as it is stored in the database
      # this updates the mongodb document with analysis result and current timestamp
      resumes_collection.update_one(
          {'_id': latest_resume['_id']},
          {
              '$set': {
                  'analysis': analysis,
                  'highlights': highlights,
                  'analyzed_at': datetime.utcnow()
              }
          }
      )

        #this lines print status messages based on the output of the process
      print("Resume analysis completed and stored in database")
  else:
      print("No text could be extracted from the resume")
else:
  print("No resume found in database")


# Print results for verification
print("\nAnalysis Result:")
print("=" * 50)
print(analysis)
# Modify the verification print section
print("\nSections to Highlight:")
print("=" * 50)
if isinstance(highlights, dict) and 'sections' in highlights:
    for section in highlights['sections']:
        print(f"Text: {section['text']}")
        print(f"Reason: {section['reason']}")
        print(f"Suggestion: {section['suggestion']}")
        print("-" * 30)

Resume analysis completed and stored in database

Analysis Result:
Kaxit Pandya is a highly skilled Computer Engineering professional with a Master's degree in Computer Engineering specializing in AI from the University of Waterloo and a Bachelor's degree from Pandit Deendayal Energy University in India. With a strong academic background in Machine Learning, Artificial Intelligence, and Data Mining, Kaxit has gained practical experience as an AI/ML Data Engineer at Genellipse Inc., where he developed text extraction pipelines and implemented AI models for data interpretation. His research internships at prestigious institutions like IIT and ISRO involved designing Deep Learning models and working on Federated Learning projects. Kaxit's technical expertise includes **Python, C, C++, JavaScript, SQL**, and libraries like **NumPy, Pandas, TensorFlow**, and frameworks such as **Node.js** and **Express**. He has also demonstrated leadership skills as a Technical Project Manager at Waterloo 

In [13]:
try:
  # Print the analysis result
  #This code block is responsible for printing the results of the resume analysis
  print("\nAPI Analysis Result:")
  print("=" * 50)
  print(analysis)
  print("=" * 50)

#any errors that occurred during the process
except Exception as e:
  error_message = f"Error analyzing resume: {str(e)}"
  print("\nError:")
  print("=" * 50)
  print(error_message)
  print("=" * 50)


API Analysis Result:
Kaxit Pandya is a highly skilled Computer Engineering professional with a Master's degree in Computer Engineering specializing in AI from the University of Waterloo and a Bachelor's degree from Pandit Deendayal Energy University in India. With a strong academic background in Machine Learning, Artificial Intelligence, and Data Mining, Kaxit has gained practical experience as an AI/ML Data Engineer at Genellipse Inc., where he developed text extraction pipelines and implemented AI models for data interpretation. His research internships at prestigious institutions like IIT and ISRO involved designing Deep Learning models and working on Federated Learning projects. Kaxit's technical expertise includes **Python, C, C++, JavaScript, SQL**, and libraries like **NumPy, Pandas, TensorFlow**, and frameworks such as **Node.js** and **Express**. He has also demonstrated leadership skills as a Technical Project Manager at Waterloo AI Institute and as the Web Development Head 

# **Scores based on orgpath pdf**

In [14]:
import json
from datetime import datetime

# Updated criteria based on the OrgInsights Assessment guide
    # "Overall Score",
    # "Develops Talent",
    # "Builds Relationships",
    # "Communicates Effectively",
    # "Drives Results",
    # "Demonstrates Adaptability",
    # "Shows Business Acumen",
    # "Thinks Strategically",
    # "Leads Change",
    # "Demonstrates Integrity"
criteria = [
    "OVERALL SCORE",
    "FORMATTING & PRESENTATION",
    "SOFT-SKILLS INFORMATION",
    "RELEVANT EXPERIENCE",
    "ACHIEVEMENT & IMPACT"

]
def generate_assessment_scores(analysis_text, criteria):
    client = OpenAI(api_key=api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that generates numerical scores for resume criteria based on the OrgInsights Assessment guide. Output should be a JSON object with scores between 0 and 100 for each criterion. "
                },
                {
                    "role": "user",
                    "content": f"Based on this resume text, generate scores for the following criteria: {', '.join(criteria)}. Analysis: {analysis_text}"
                }
            ],
            temperature=0
        )

        content = response.choices[0].message.content.strip()
        print("Raw API response:", content)  # Print raw response for debugging
        raw_response = content.strip()
        if raw_response.startswith("```json") and raw_response.endswith("```"):
            content = raw_response[7:-3].strip()

        

        if content.startswith("``````"):
            content = content[3:-3]
        if content.startswith("json"):
            content = content[4:]
        content = content.strip()

        scores = json.loads(content)
        return scores

    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {str(e)}")
        print("Raw content causing error:", content)

        return f"Error parsing JSON: {str(e)}"
    except Exception as e:
        print(f"General Error: {str(e)}")
        return f"Error generating assessment scores: {str(e)}"

# Assuming you have the analysis text stored in a variable called 'analysis'
scores = generate_assessment_scores(analysis, criteria)

if isinstance(scores, dict):
    # Store the analysis and scores back in MongoDB
    resumes_collection.update_one(
        {'_id': latest_resume['_id']},
        {
            '$set': {
                'assessment_scores': scores,
                'scored_at': datetime.utcnow()
            }
        }
    )

    print("Resume analysis and assessment scores completed and stored in database")

    print("\nAPI Analysis Result:")
    print("=" * 50)
    print(analysis)
    print("\nAssessment Scores:")
    print("=" * 50)

    for criterion, score in scores.items():
        print(f"{criterion}: {score}")
    print("=" * 50)

    # Add interpretation of scores based on the guide
    print("\nScore Interpretation:")
    print("=" * 50)
    experience_level = "Entry-level"  # You may need to determine this based on the resume content
    for criterion, score in scores.items():
        if criterion != "Overall Score":
            if experience_level == "Entry-level":
                if score >= 70:
                    print(f"{criterion}: Excellent for entry-level")
                elif score >= 50:
                    print(f"{criterion}: Good for entry-level")
                else:
                    print(f"{criterion}: Needs improvement")
            else:
                if score >= 80:
                    print(f"{criterion}: Meets senior-level expectations")
                elif score >= 60:
                    print(f"{criterion}: Acceptable, but room for growth")
                else:
                    print(f"{criterion}: Below expectations for experienced professionals")
    print("=" * 50)

else:
    print("Error in generating scores:")
    print(scores)


Raw API response: ```json
{
    "OVERALL SCORE": 85,
    "FORMATTING & PRESENTATION": 90,
    "SOFT-SKILLS INFORMATION": 80,
    "RELEVANT EXPERIENCE": 85,
    "ACHIEVEMENT & IMPACT": 90
}
```
Resume analysis and assessment scores completed and stored in database

API Analysis Result:
Kaxit Pandya is a highly skilled Computer Engineering professional with a Master's degree in Computer Engineering specializing in AI from the University of Waterloo and a Bachelor's degree from Pandit Deendayal Energy University in India. With a strong academic background in Machine Learning, Artificial Intelligence, and Data Mining, Kaxit has gained practical experience as an AI/ML Data Engineer at Genellipse Inc., where he developed text extraction pipelines and implemented AI models for data interpretation. His research internships at prestigious institutions like IIT and ISRO involved designing Deep Learning models and working on Federated Learning projects. Kaxit's technical expertise includes **Pyth

**Elevator Pitch**

In [15]:
import json
import openai
from datetime import datetime, timezone

def generate_elevator_pitch(resume_text, api_key):
    client = OpenAI(api_key=api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that creates concise and impactful elevator pitches based on resume information. The elevator pitch should be no more than 1 minute long when spoken, highlighting the candidate's key strengths, experiences, and unique value proposition."
                },
                {
                    "role": "user",
                    "content": f"Based on the following resume, create an elevator pitch for him/her, write in 1st person tone:\n\n{resume_text}"
                }
            ],
            temperature=0.7
        )

        elevator_pitch = response.choices[0].message.content.strip()
        return elevator_pitch

    except Exception as e:
        return f"Error generating elevator pitch: {str(e)}"


resume_text = text

# Generate the elevator pitch
try:
    elevator_pitch = generate_elevator_pitch(resume_text, api_key)

    # Store the elevator pitch in MongoDB (assuming you have a MongoDB connection set up)
    resumes_collection.update_one(
        {'_id': latest_resume['_id']},
        {
            '$set': {
                'elevator_pitch': elevator_pitch,
                'pitch_generated_at': datetime.now(timezone.utc)
            }
        }
    )

    print("Elevator pitch generated and stored successfully:")
    print(elevator_pitch)
except Exception as e:
    print(f"Error: {str(e)}")


Elevator pitch generated and stored successfully:
Hello, I'm Kaxit Pandya, a Master of Engineering in Computer Engineering with a specialization in AI from the University of Waterloo. With a strong background in AI and ML, I've developed innovative solutions like a text extraction pipeline using BERT models at Genellipse Inc. I've also excelled in research, implementing GANs for material design at IIT Gandhinagar and FL models for water quality prediction in Taiwan. My expertise spans from software and web development to deep learning and cloud services. Let's connect to bring cutting-edge AI solutions to the forefront of technology!


In [16]:
star_text='''Pre-Interview Preparation: What Every Job Seeker And New Graduate Should Do Before The First Phone Screen Or Interview
Before you even get on a phone call or step into an interview room, proper preparation is key to ensuring you make a strong impres-sion. Taking the time to prepare not only helps you feel more confi-dent, but it also shows potential employers that you are serious about the opportunity.
Research the Company Thoroughly
Before any interaction with the hiring team, it's critical to research the company. A lack of knowledge about the employer can hurt your chances, as it suggests you’re uninterested or unprepared. Here’s what you need to focus on:
• Company Overview: Familiarize yourself with the company’s mission, values, and culture. Check their website, social media channels, and any news articles to stay informed about recent developments.
• Industry: Understand the company’s position in its industry and its competitors. You don’t need to be an expert, but having general knowledge shows you’re serious about the opportunity.
• Key Players: Look up the leadership team, especially anyone who might be interviewing you. Knowing who you’ll be speaking with can give you an edge and help you tailor your questions or conversation.
Study the Job Description in Detail
A well-prepared candidate knows the job description inside and out. Break down the requirements and make sure you understand what’s expected of the role. Match your skills, experience, and achievements to the key qualifications they are looking for.
• Identify your strengths: Highlight areas where your skills perfectly align with the job description.
• Prepare to address any gaps: If you don’t meet 100% of the qualifications, be ready to discuss how your other skills or your ability to learn quickly can bridge those gaps.
Prepare Your Key Talking Points
You’ll want to walk into the interview ready to discuss your background, but it’s important to focus on what’s relevant to the position. Here’s how to prepare:
• Craft your elevator pitch: Prepare a concise summary of who you are, your professional background, and what you bring to the role.
• Frame your experience: Think about 3-5 accomplishments or projects that best demonstrate your abilities and tie them directly to the job you’re applying for.
LEVELLING UP YOUR INTERVIEW GAME:
BECOMING A STAR
15
Pre-Interview Preparation: What Every Job Seeker And New Graduate Should Do Before The First Phone Screen Or Interview Cont...
Prepare Thoughtful Questions
Hiring managers are evaluating you, but you should also be evaluating the company. Prepare questions that not only demonstrate your interest in the role but also help you gauge whether the company is the right fit for you. Examples include:
“What are the key goals for someone in this role in the first 6-12 months?”
“Can you tell me about the company culture and team dynamics?”
“What does success look like in this position?”
Asking thoughtful questions can show your genuine interest in the role, while also giving you deeper in-sight into the position and the company.
Test Your Technology for Virtual Interviews
If your interview is going to be virtual or over the phone, don’t wait until the last minute to check your equipment. Technical issues can quickly derail a great conversation and leave a poor impression.
• Test your internet connection: Make sure it’s stable and reliable.
• Test your camera and microphone: For video interviews, ensure the camera is positioned well, with good lighting, and that your microphone is clear.
• Find a quiet space: Minimize background noise, and ensure that your surroundings are professional and free from distractions.
Review Your Resume and LinkedIn Profile
Make sure your resume is up-to-date and consistent with what’s on your LinkedIn profile. Hiring managers often check both. Be prepared to talk through any gaps in employment, transitions between jobs, or spe-cific projects you’ve highlighted. Also, make sure your LinkedIn profile is polished and reflects the same key points as your resume.
Practice Mock Interviews
Whether it’s with a friend, mentor, or even in front of a mirror, mock interviews help you polish your re-sponses and get comfortable speaking about your experience. Focus on:
• Clear, concise answers: Don’t ramble or go off-topic.
• Body language: Make sure to maintain eye contact, sit up straight, and use appropriate gestures.
• Pacing and tone: Avoid speaking too fast or too slowly. Practice maintaining an upbeat, confident tone.
Know Your Availability and Salary Expectations
Be prepared to discuss your availability for the role, including your potential start date. If salary is likely to come up, have a range ready based on your research of the market and your level of experience. Sites like Glassdoor and PayScale can help you find industry-standard salaries for similar positions.
Get a Good Night’s Sleep
It might seem obvious, but getting enough sleep the night before is essential for being at your best. A well-rested mind will help you stay calm, focused, and responsive during your interview.
16
Dress For Success - Interview Attire and Etiquette
When it comes to interviews, both in-person and virtual, the way you present yourself can make a lasting impression. Whether you're a seasoned professional or a fresh graduate, dressing and be-having appropriately plays a crucial role in shaping how employers perceive you.
In-Person Interviews:
• Understand the Company Culture: Research the company's dress code beforehand. If it's a cor-porate or conservative environment, opt for formal attire (e.g., suits or professional dresses). For more relaxed workplaces, business casual is acceptable, but it's better to be slightly overdressed than underdressed.
• Invest in Fit and Quality: Your clothing should be clean, well-fitted, and wrinkle-free. Ill-fitting clothes can be distracting and suggest a lack of attention to detail. Polish your shoes and avoid flashy or excessive accessories.
• Neutral and Professional Colors: Stick to neutral colors like navy, black, gray, or white. These con-vey professionalism and are less likely to distract the interviewer from your qualifications and skills.
• Grooming Matters: Personal hygiene is paramount. Make sure your hair is neat, your nails are clean, and your breath is fresh. Avoid overpowering colognes or perfumes, as these can be dis-tracting or bothersome in a small room.
Online Interviews:
• Professional from the Waist Up: Even if you're behind a screen, dress as if you’re meeting in per-son. At the very least, wear a button-up shirt, blouse, or blazer. Avoid the temptation to wear sweatpants or pajamas below the camera line—you never know when you'll need to stand up!
• Choose a Simple Background: Your interview setting should be distraction-free. A plain wall or a tidy space will keep the focus on you, not your surroundings. Ensure proper lighting, and sit facing a window or a light source for a clear, bright appearance.
• Test Technology in Advance: Dress rehearsal isn't just for actors—test your equipment (camera, mic, internet) before the interview to avoid any last-minute hiccups. It shows you're prepared and serious about the opportunity.
17
How To Highlight Soft Skills in An Interview
The interview is the time to show off those soft skills in action. Here’s how you can subtly weave them in-to your answers:
Use the STAR Method to Show Problem-Solving & Teamwork
The STAR method (Situation, Task, Action, Result) is perfect for soft skills questions. It gives you a struc-ture for storytelling that really highlights how you’ve used your skills in real life.
Example Question: “Tell me about a time you worked in a team.”
Answer using STAR:
• Situation: “At my internship, we had to work on a project with a tight deadline.”
• Task: “I was responsible for coordinating between the marketing and sales teams to ensure every-one was on the same page.”
• Action: “I set up weekly meetings and clear communication channels to keep everyone aligned.”
Result: “We finished the project ahead of schedule, and our cross-team collaboration increased by 25%.”
This shows teamwork, communication, and time management—all in one story! Plus, it paints you as a capable team player with real results.
When They Ask About Strengths, Highlight Soft Skills
When the interviewer says, “Tell me about your strengths,” it’s your time to shine. Don’t just say you’re a “good communicator”—talk about how you communicate well.
Example Answer: “I think one of my biggest strengths is my ability to communicate with people from different backgrounds. For example, at my last job, I regularly had to explain technical information to non-technical team members, and I got really good at simplifying complex concepts so everyone understood.”
This way, you’re proving the strength instead of just saying it.
Answer Behavioral Questions with Soft Skills in Mind
Behavioral questions are designed to find out how you’ll handle situations at work. Always answer these with soft skills at the forefront.
Example Behavioral Question: “How do you handle stress or tight deadlines?”
Answer: “I prioritize tasks and focus on what’s most urgent first. For example, during finals week at school, I had to juggle multiple assignments. I broke them down into smaller steps and communicated with my professors about expectations. By staying organized and calm, I was able to meet all my dead-lines without sacrificing quality.”
You’ve just demonstrated adaptability, time management, and communication in one answer.
'''

In [17]:
def generate_star(star_text, text):
    client = OpenAI(api_key=api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": f'''Fetch information about STAR method, This is some star information:\n\n {star_text} \n\n You are a STAR (Situation, Task, Action, Result) master. You give star information by reviewing their resume. Give result without any starting or ending statements, just in different points. Also dont add any "-" in between. Format the output so that the words **Situation**, **Task**, **Action**, and **Result** are bold using markdown syntax .'''
                },
                {
                    "role": "user",
                    "content": f"This is the resume text:\n\n {text} \n\n analyse it and identify specific situations, tasks, actions, and results from it that can be used during an interview. Clearly label each STAR component and provide 3 STAR examples:"
                }
            ],
            temperature=0.7                                                                   
        )

        star_info = response.choices[0].message.content.strip()
        return star_info

    except Exception as e:
        return f"Error generating elevator pitch: {str(e)}"


try:
    star_info = generate_star(star_text, text)

    # Store the elevator pitch in MongoDB (assuming you have a MongoDB connection set up)
    resumes_collection.update_one(
        {'_id': latest_resume['_id']},
        {
            '$set': {
                'star_info': star_info,
                'star_generated_at': datetime.now(timezone.utc)
            }
        }
    )

    print("STAR INFO generated and stored successfully:")
    print(star_info)
except Exception as e:
    print(f"Error: {str(e)}")


STAR INFO generated and stored successfully:
**Situation**: Working as an AI/ML Data Engineer at Genellipse Inc.  
**Task**: Building a text extraction pipeline for processing diverse document formats.  
**Action**: Leveraging Google Cloud Vision API to extract data from U.S. death certificates with high accuracy.  
**Result**: Implementing BERT-based AI models to map key-value pairs accurately and delivering actionable insights for enhanced client reporting.

**Situation**: Research Intern at Indian Space Research Organisation (ISRO), SAC  
**Task**: Developing Deep Neural Networks for predicting classification of Pulse Radar Interval Modulation.  
**Action**: Processing and handling noisy, raw radar pulse data directly from ISRO’s satellites.  
**Result**: Achieving 94% classification accuracy in identifying complex radar Pulse Repetition Intervals (PRI) patterns.

**Situation**: Technical Project Manager at Waterloo AI Institute (WAT.AI)  
**Task**: Building deployable AI models on 